# Data analysis project

_In this project I am going to write a program to predict the number likes post is going to get in vk._

__Выполнил__: Булгаков Дмитрий (ИАД16)

__Дедлайн__: 23:59 10.04.16

## 1. Loading Data

VK library quiet installation and import into the notebook.

In [4]:
# !pip install vk # makes it quiet
import vk

Starting new vk session in order to parse data

In [5]:
vk_session = vk.Session() # starting new session
vk_api = vk.API(vk_session)

Getting number of posts in selected vk group.

In [6]:
selected_group = 'overhearhse' # no other ideas :c
posts_number = vk_api.wall.get(domain=selected_group)[0] # number of posts is stored in first element
print('Number of posts in selected group: ', posts_number - 1)

Number of posts in selected group:  13125


Writing a function to parse more, than 100 posts from group.

In [7]:
def load_all_posts(page, n_posts, api):
    all_posts = api.wall.get(domain=page, count=n_posts)
    n_loaded = len(all_posts)
    while n_loaded < n_posts: # loop to load more, than 100 posts
        s = api.wall.get(domain=page, offset=n_loaded, count=(n_posts - n_loaded)) # update offset
        all_posts += s[1:] # no need for first element
        n_loaded += len(s) - 1 # update n_loaded
    return all_posts

Loading all posts from group for future analysis

In [32]:
try:
    loaded_posts = load_all_posts(page=selected_group, n_posts=1501, api=vk_api)[1:] # no need for posts number element
    # 1500 for this time, because I have small amount of ram avaliable :c
    print('Number of loaded posts: ', len(loaded_posts))
except: # timout errors are often to occur
    print('Error occured! Try again.')

Number of loaded posts:  1500


## 2. Data preprocessing

Loading required libs to preprocess data.

In [33]:
# !pip install pymorphy2 -q # silent install again
# !pip install stop_words -q # needed to remove stop words
from stop_words import get_stop_words
import pymorphy2 # need this one to convert words to normal time
import datetime # needed to convert response date 
import string # needed to work with strings
from nltk.tokenize import TweetTokenizer # needed to split text
import pandas as pd # required to work with dataframes
from ipywidgets import IntProgress # progressbar
from IPython.display import display # progressbar

Writing functions to process text data. Converting words to normal form and removing punctuation here.

In [34]:
def split_text(text):
    tokenizer = TweetTokenizer()
    return tokenizer.tokenize(text) # spliting text into words

def convert_to_normal_form(words_list):
    morph = pymorphy2.MorphAnalyzer()
    normal_forms_list = []
    for word in words_list:
        if word not in string.punctuation and word[0] != "<":
            norm_form = morph.parse(word)[0].normal_form #getting normal form of a word
            normal_forms_list.append(norm_form) #adding it to list
    return normal_forms_list

def convert_text(text):
    words_list = split_text(text) # spliting text into words
    norm_words_list = convert_to_normal_form(words_list) # words into normal form
    filtered_words = [w for w in norm_words_list if not w in get_stop_words('russian')] # removing stop words
    return " ".join(filtered_words) # joining words to a sentence again

Writing a function to convert received list into another with another data.

In [35]:
def convert_posts(posts_list):
    progress = IntProgress() 
    progress.max = len(posts_list) # initializing progressbar
    progress.description = 'Processing data convertion'
    display(progress)
    
    updated_posts = [] # list of new posts' list structure
    for i, post in enumerate(posts_list): 
        tmp_dict = {} # creating empty dictionary for each post
        tmp_dict['likes_number'] = int(post['likes']['count']) # getting likes count
        tmp_dict['text'] = convert_text(post['text']) # converting text into normal form
        tmp_dict['text_length'] = len(post['text']) # calculating text length
        tmp_dict['post_hour'] = int(datetime.datetime.fromtimestamp(post['date']).strftime('%H')) # parsing only post hour
        tmp_dict['post_month'] = int(datetime.datetime.fromtimestamp(post['date']).strftime('%m')) # and post month
        tmp_dict['signed'] = int(post['from_id'] != -57354358) # checking whether post is signed or not
        # checking if any attacment exists
        tmp_dict['with_attachment'] = 1 if 'attachment' in post.keys() else 0
        tmp_dict['pinned'] = 1 if 'is_pinned' in post.keys() else 0 # cheking if post is pinned
        tmp_dict['repost'] = 1 if post['post_type'] == 'copy' else 0 # cheking if repost
        updated_posts.append(tmp_dict)
        progress.value += 1 # increasing progressbar value
    progress.description = 'Done convertion!'
    return updated_posts

Converting list of posts into new more convenient one.

In [36]:
converted_posts = convert_posts(loaded_posts)

## 3. Creating object-feature matrix

Loading pandas

In [37]:
import pandas as pd

Creating dataframe from parsed data

In [38]:
posts_frame = pd.DataFrame(converted_posts)
posts_frame.head()

,likes_number,pinned,post_hour,post_month,repost,signed,text,text_length,with_attachment
0,86,1,13,4,0,0,,0,1
1,0,0,14,4,0,0,6 собраться валить юрфак спбгу мол учиться хор...,208,0
2,92,0,9,4,0,0,,0,1
3,6,0,9,4,0,0,мгу приходить крутой актёр телеведущий собират...,148,0
4,7,0,20,4,1,0,восхищать удивительный недееспособность больши...,410,0


And describing posts data

In [39]:
posts_frame.describe()

,likes_number,pinned,post_hour,post_month,repost,signed,text_length,with_attachment
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.000000,1500.000000
mean,45.812000,0.000667,15.220667,6.093333,0.034667,0.0,250.044000,0.358667
std,56.131701,0.025820,5.666749,4.238467,0.182995,0.0,541.126328,0.479769
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000
25%,10.000000,0.000000,12.000000,2.000000,0.000000,0.0,48.000000,0.000000
50%,28.000000,0.000000,16.000000,7.000000,0.000000,0.0,115.000000,0.000000
75%,62.000000,0.000000,20.000000,10.000000,0.000000,0.0,242.500000,1.000000
max,581.000000,1.000000,23.000000,12.000000,1.000000,0.0,8014.000000,1.000000


Creating object-feature matrix

In [40]:
from sklearn.feature_extraction.text import CountVectorizer # loading count vectorizer
cv = CountVectorizer()
train_features = cv.fit_transform(posts_frame['text']).toarray() # vectorizing texts
train_frame = posts_frame.join(pd.DataFrame(train_features, columns=cv.get_feature_names())) # transfering it to pandas
train_frame.drop(['likes_number','text'],inplace=True,axis=1,errors='ignore') # removing unnecessary columns
train_frame

,pinned,post_hour,post_month,repost,signed,text_length,with_attachment,00,000,00000002,...,яшин,ящик,ящитая,ёлка,ёлочка,ёмкий,ёпть,ёршик,ёт,қант
0,1,13,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,14,4,0,0,208,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,9,4,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,9,4,0,0,148,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,20,4,1,0,410,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,16,4,0,0,163,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,13,4,0,0,79,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,13,4,0,0,467,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,22,4,0,0,215,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,13,4,0,0,87,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
train_frame.describe()

,pinned,post_hour,post_month,repost,signed,text_length,with_attachment,00,000,00000002,...,яшин,ящик,ящитая,ёлка,ёлочка,ёмкий,ёпть,ёршик,ёт,қант
count,1500.000000,1500.000000,1500.000000,1500.000000,1500.0,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,...,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000,1500.000000
mean,0.000667,15.220667,6.093333,0.034667,0.0,250.044000,0.358667,0.003333,0.005333,0.000667,...,0.000667,0.000667,0.000667,0.002667,0.000667,0.000667,0.000667,0.000667,0.000667,0.000667
std,0.025820,5.666749,4.238467,0.182995,0.0,541.126328,0.479769,0.068254,0.089313,0.025820,...,0.025820,0.025820,0.025820,0.051588,0.025820,0.025820,0.025820,0.025820,0.025820,0.025820
min,0.000000,0.000000,1.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,12.000000,2.000000,0.000000,0.0,48.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,16.000000,7.000000,0.000000,0.0,115.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,20.000000,10.000000,0.000000,0.0,242.500000,1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,23.000000,12.000000,1.000000,0.0,8014.000000,1.000000,2.000000,2.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


Saving train frame to file

In [42]:
train_frame.to_csv('traindata.csv')

To be continued....